# 一些用不到的

In [6]:
import h5py
import pandas as pd

## 查看 wifi csi 的数据格式

In [7]:

file_path = "D:/MyCodes/all_30_3/train_data.h5"


with h5py.File(file_path, 'r') as f:
    print("Keys in the file:", list(f.keys()))

f = h5py.File(file_path, 'r')






Keys in the file: ['imu', 'wifi']


In [3]:
label = f["label"]
print(label[:])

[[   1   24    0  250]
 [   2    7  250  650]
 [   3   22  650 1350]
 [   4    1 1350 1750]
 [   5    5 1750 2349]
 [   6   31 2349 2749]
 [   7   21 2749 3400]
 [   8   29 3400 3700]]


In [4]:
def process_wifi(self, sample_data, sample_label, target_data_len=4096, target_label_len=12):
        """
        input: 
            data: seq_len * 3 * 3 * 30
            label: n * 4
        
        output:
            data: (3 * 3 * 30) * target_data_len
            label: target_label_len * start * end * action
        """

        # if seq_len = target_data_len:
        ret_data = sample_data
        ret_label_1 = sample_label
        ret_label_2 = sample_data

        duration = sample_data.size(0)
        num_actions = sample_label.size(0)
        pad_size = 0

        # 截取或补充 data 并更新 label
        if duration < target_data_len:
             pad_size = target_data_len - duration
             pad_tensor = torch.zeros((pad_size, *ret_data.shape[1:]), device=ret_data.device)
             ret_data = torch.cat([ret_data, pad_tensor], dim = 0)
            # TO DO update label
             pad_label_tensor = torch.tensor([[num_actions + 1, 0, duration, target_data_len]], dtype=sample_label.dtype, device=sample_label.device)
             ret_label_1 = torch.cat([ret_label_1, pad_label_tensor], dim=0)
        else:
             # 截取 data
            ret_data = ret_data[:target_data_len, :, :, :]

            # 筛选出完全超出范围的动作（start >= target_label_len）
            valid_mask_discard = (ret_label_1[:, 2] < target_label_len)
            ret_label_1 = ret_label_1[valid_mask_discard]

            # 处理部分超出范围的动作（start < target_label_len 且 end >= target_label_len）
            valid_mask_truncate = (ret_label_1[:, 2] < target_label_len) & (ret_label_1[:, 3] >= target_label_len)
            ret_label_1[valid_mask_truncate, 3] = target_label_len - 1  # 截断 end

        
        # 补充 label 到固定长度，便于生成dataset, 一般 label 数量只会小于规定的label长度
        num_actions = ret_label_1.size(0)
        if  num_actions < target_label_len:
            need = target_label_len - num_actions
            for i in range(1, need + 1):
                pad_tensor = torch.tensor(
                    [[num_actions + i, 0, ret_data.size(0), ret_data.size(0)]],
                    dtype=sample_label.dtype, device=sample_label.device
                )
                ret_label_1 = torch.cat([ret_label_1, pad_tensor], dim=0)
        
        # 调整形状
        ret_label_1 = ret_label_1[:, 1:]
        ret_data = ret_data.permute(3, 1, 2, 0)
        ret_data = ret_data.reshape(-1, ret_data.shape[-1])  # [3*3*30, 2048]
        ret_label_1 = ret_label_1[:, [1, 2, 0]]

        # 归一化
        if self.normalize:
            # 实现归一化逻辑
            pass

        return ret_data, ret_label_1

## 测试wifi模态

In [5]:
import numpy as np

npy_file_path = r'D:\MyCodes\wifi-tad-data\dataset\smartwifi\validation_npy\1_sjz_0.npy'

data = np.load(npy_file_path)

# 打印数据
print(data.shape)


(8500, 30)


In [6]:
import os
import h5py
import torch
from torch.utils.data import Dataset
import torch.nn.functional as F



class DatasetSingle(Dataset):
    def __init__(self, dataset_dir="", split="train", normalize=False, modality="wifi", device_keep_list=None):
        super().__init__()
        assert split in ["train", "test"], "split must be 'train' or 'test'"
        self.dataset_dir = dataset_dir
        self.split = split
        self.normalize = normalize
        self.device_keep_list = device_keep_list

        # 设置数据文件夹路径
        self.folder_path = "D:/MyCodes/data/processed_data/wifi"
        if not os.path.exists(self.folder_path):
            raise FileNotFoundError(f"Folder {self.folder_path} does not exist.")

        # 获取所有 .h5 文件
        self.data_list = [f for f in os.listdir(self.folder_path) if f.endswith('.h5')]

    def __len__(self):
        return len(self.data_list)
    
   
    def process_wifi(self, sample_data, sample_label, target_data_len=4096, target_label_len=12):
        """
        input: 
            data: seq_len * 3 * 3 * 30
            label: n * 4
        
        output:
            data: (3 * 3 * 30) * target_data_len
            label: target_label_len * start * end * action
        """

        # if seq_len = target_data_len:
        ret_data = sample_data
        ret_label_1 = sample_label
        ret_label_2 = sample_data

        duration = sample_data.size(0)
        num_actions = sample_label.size(0)
        pad_size = 0

        # 截取或补充 data 并更新 label
        if duration < target_data_len:
             pad_size = target_data_len - duration
             pad_tensor = torch.zeros((pad_size, *ret_data.shape[1:]), device=ret_data.device)
             ret_data = torch.cat([ret_data, pad_tensor], dim = 0)
            # TO DO update label
             pad_label_tensor = torch.tensor([[num_actions + 1, 0, duration, target_data_len]], dtype=sample_label.dtype, device=sample_label.device)
             ret_label_1 = torch.cat([ret_label_1, pad_label_tensor], dim=0)
        else:
             # 截取 data
            ret_data = ret_data[:target_data_len, :, :, :]

            # 筛选出完全超出范围的动作（start >= target_label_len）
            valid_mask_discard = (ret_label_1[:, 2] < target_label_len)
            ret_label_1 = ret_label_1[valid_mask_discard]

            # 处理部分超出范围的动作（start < target_label_len 且 end >= target_label_len）
            valid_mask_truncate = (ret_label_1[:, 2] < target_label_len) & (ret_label_1[:, 3] >= target_label_len)
            ret_label_1[valid_mask_truncate, 3] = target_label_len - 1  # 截断 end

        
        # 补充 label 到固定长度，便于生成dataset, 一般 label 数量只会小于规定的label长度
        num_actions = ret_label_1.size(0)
        if  num_actions < target_label_len:
            need = target_label_len - num_actions
            for i in range(1, need + 1):
                pad_tensor = torch.tensor(
                    [[num_actions + i, 0, ret_data.size(0), ret_data.size(0)]],
                    dtype=sample_label.dtype, device=sample_label.device
                )
                ret_label_1 = torch.cat([ret_label_1, pad_tensor], dim=0)
        
        # 调整形状
        ret_label_1 = ret_label_1[:, 1:]
        ret_data = ret_data.permute(3, 1, 2, 0)
        ret_data = ret_data.reshape(-1, ret_data.shape[-1])  # [3*3*30, 2048]
        ret_label_1 = ret_label_1[:, [1, 2, 0]]

        # 实现 ancher-free 的 label
        # ret_label_2 = -1 * np.ones(target_data_len, dtype=int)
        # for start_time, end_time, action in ret_label_1:
        #     ret_label_2[start_time:end_time] = action
        #     # TODO
            # ret_label_2[start_time] = 
            # ret_label_2[end_time] = 

        # 归一化
        if self.normalize:
            # 实现归一化逻辑
            pass

        return ret_data, ret_label_1
    
    # target_len = 4096
    # def process_wifi(self, sample_data, sample_label, target_data_len=4096, target_label_len=12):
    #     """
    #     input: 
    #         data: t * 3 * 3 * 30
    #         label: n * 4
        
    #     output:
    #         data: channel * target_data_len * 3 * 3
    #         label: target_label_len * start * end * action
    #     """

    #     # 补充 data 到固定长度，便于生成dataset
    #     if sample_data.size(0) < target_data_len:
    #         duration = sample_data.size(0)
    #         pad_size = target_data_len - duration
    #         # 确保 pad_data_tensor 的形状与 sample_data 一致
    #         pad_data_tensor = torch.zeros((pad_size, *sample_data.shape[1:]), device=sample_data.device)
    #         ret_data = torch.cat([sample_data, pad_data_tensor], dim=0)

    #         # 补充 label
    #         # num_actions = sample_label.size(0)
    #         # pad_label_tensor = torch.tensor([[num_actions + 1, 0, duration, target_data_len]], dtype=sample_label.dtype, device=sample_label.device)
    #         # ret_label = torch.cat([sample_label, pad_label_tensor], dim=0)
    #         ret_label = sample_label
    #     else:
    #         ret_data = sample_data
    #         ret_label = sample_label

    #     # 补充 label 到固定长度，便于生成dataset
    #     if ret_label.size(0) < target_label_len:
    #         num_actions = ret_label.size(0)
    #         need_nums_label = target_label_len - num_actions
    #         offset = (target_data_len - sample_data.size(0)) / need_nums_label if need_nums_label > 0 else 0
    #         ret_label_2 = ret_label  # 使用 ret_label 而不是 sample_label
    #         for i in range(1, need_nums_label + 1):  # 修复语法错误
    #             pad_tensor = torch.tensor(
    #                 [[num_actions + i, 0, sample_data.size(0) + (i - 1) * offset, sample_data.size(0) + i * offset]],
    #                 dtype=sample_label.dtype, device=sample_label.device
    #             )
    #             ret_label_2 = torch.cat([ret_label_2, pad_tensor], dim=0)
    #     else:
    #         ret_label_2 = ret_label

    #     ret_label_2 = ret_label_2[:, 1:]

    #     # 调整形状至与 wifi tad 的匹配
    #     ret_data = ret_data.permute(3, 1, 2, 0)
    #     ret_data = ret_data.reshape(-1, ret_data.shape[-1])  # [3*3*30, 2048]
    #     ret_label_2 = ret_label_2[:, [1, 2, 0]]

    #     # 归一化
    #     if self.normalize:
    #         # 实现归一化逻辑
    #         pass

    #     return ret_data, ret_label_2

    
    def __getitem__(self, index):
        if torch.is_tensor(index):
            index = index.tolist()

        try:
            data_name = self.data_list[index]
            data_path = os.path.join(self.folder_path, data_name)

            with h5py.File(data_path, 'r') as f:
                # 加载数
                data_sample = f['amp'][()]  # 假设幅值数据存储在 'amp' 键下
                label_sample = f['label'][()]  # 假设标签数据存储在 'label' 键下

                # 转换为 PyTorch 张量
                sample_data_tensor = torch.tensor(data_sample, dtype=torch.float32)
                sample_label_tensor = torch.tensor(label_sample, dtype=torch.float32)

                # 处理 wifi 信号
                data, label = self.process_wifi(sample_data_tensor, sample_label_tensor)

                return data, label
        except Exception as e:
            print(f"Error loading file {data_path}: {e}")
            raise



In [56]:
from torch.utils.data import DataLoader

dataset = DatasetSingle()
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

# 将 DataLoader 转换为迭代器
data_iter = iter(dataloader)

# 获取一个批次
data, label = next(data_iter)
print("Data shape:", data.shape)
print("Label shape:", label.shape)

print(label)

NameError: name 'DatasetSingle' is not defined

# 载入实际训练时的数据

In [1]:
import sys
# sys.path.append(r'D:\MyCodes\XRF\XRFV2-main')
sys.path.append(r'D:\MyCodes\CLIPBased_TAD')
from dataset.wwadl import WWADLDatasetSingle, detection_collate
from torch.utils.data import DataLoader
from dataset.wwadl_multi import WWADLDatasetMutiAll



In [2]:
# train_dataset = WWADLDatasetSingle(r'D:\MyCodes\all_30_3', split='train', modality='wifi')
train_dataset_multi = WWADLDatasetMutiAll(r'D:\MyCodes\all_30_3', split='train')

In [3]:
 # 定义 DataLoader
batch_size = 10
train_data_loader = DataLoader(
    train_dataset_multi,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,
    collate_fn=detection_collate,
    pin_memory=True,
    drop_last=True
)

In [4]:
# 将 DataLoader 转换为迭代器
data_iter = iter(train_data_loader)

# 获取一个批次
data, label = next(data_iter)


# 尝试载入 wifi_tad 模型进行训练


In [5]:
import sys
sys.path.append(r'D:\MyCodes\WiFi_TAD\WiFiTAD_main')


In [6]:
from TAD.model.tad_model import wifitad
from TAD.config import config

In [7]:
batch_size = config['training']['batch_size']
learning_rate = config['training']['learning_rate']
weight_decay = config['training']['weight_decay']
max_epoch = config['training']['max_epoch']
num_classes = config['dataset']['num_classes']
checkpoint_path = config['training']['checkpoint_path']
focal_loss = config['training']['focal_loss']
random_seed = config['training']['random_seed']
ngpu = config['ngpu']
GLOBAL_SEED = 1

In [8]:
import torch
net = wifitad(in_channels=config['model']['in_channels'])

optimizer = torch.optim.Adam(net.parameters(),
                                 lr=learning_rate,
                                 weight_decay=weight_decay)


In [9]:
output_dict = net(data['wifi'])
output_dict['loc']

embedding shape: torch.Size([4, 512, 2048])


tensor([[[1.5074, 2.1209],
         [0.6845, 2.2998],
         [0.3900, 2.3040],
         ...,
         [0.7362, 0.9828],
         [0.7010, 1.5912],
         [0.5631, 0.6824]],

        [[0.7282, 1.6125],
         [0.9122, 0.5242],
         [0.9626, 5.7588],
         ...,
         [0.3029, 1.3716],
         [1.1825, 1.2195],
         [0.7359, 0.9288]],

        [[0.5539, 1.9842],
         [1.2792, 0.8339],
         [0.3298, 1.8912],
         ...,
         [1.0266, 0.7839],
         [0.7400, 1.6305],
         [0.7661, 0.5536]],

        [[0.5491, 2.0223],
         [1.8312, 1.0682],
         [1.8809, 1.5946],
         ...,
         [0.8926, 1.3192],
         [0.5554, 1.0669],
         [1.3258, 0.9371]]], grad_fn=<CatBackward0>)

# 测试模型的各个模块

## 测试 Embedding 模块， Embedding 使用 XRF 的模块

In [5]:
from model.XRF.embedding import TADEmbedding

In [6]:
embed = TADEmbedding(270)

In [7]:
embedding_out_wifi = embed(data['wifi'])

In [8]:
embedding_out_wifi.shape

torch.Size([4, 512, 256])

In [10]:
embed_imu = TADEmbedding(30)

In [11]:
embedding_out_imu = embed_imu(data['imu'])

In [12]:
embedding_out_imu.shape

torch.Size([4, 512, 256])

## 测试 Fusion 模块， Fusion 使用 XRF 的特征融合模块

In [13]:
from model.XRF.fusion import GatedFusionAdd2 

In [ ]:
import torch
import torch.nn as nn
class GatedFusionAdd2(nn.Module):
    def __init__(self, hidden_size: int):
        """
        使用门控机制的模态融合模块。
        Args:
            hidden_size (int): 特征的隐藏维度 (C)。
        """
        super(GatedFusionAdd2, self).__init__()
        # print('hidden_size', hidden_size)
        self.gate_linear = nn.Linear(hidden_size, hidden_size * 2)  # 1D 卷积等价于线性层
        self.gate = nn.Sigmoid()  # 门控激活函数
        self.fc = nn.Linear(hidden_size * 2, hidden_size)  # 融合后特征映射回 hidden_size

    def forward(self, imu_features: torch.Tensor, wifi_features: torch.Tensor) -> torch.Tensor:
        """
        前向传播逻辑。
        Args:
            imu_features (torch.Tensor): IMU 模态特征，形状 (B, C, L)。
            wifi_features (torch.Tensor): WiFi 模态特征，形状 (B, C, L)。
        Returns:
            torch.Tensor: 融合后的特征，形状 (B, C, L)。
        """
        # 计算门控值
        # print(imu_features.shape, wifi_features.shape)
        # print(combined_features.shape)
        # x = 
        
        gate = self.gate(self.gate_linear(imu_features + wifi_features))
        # print("hihihi", gate.shape)
        combined = torch.cat([imu_features, wifi_features], dim=-1)

        fused_features = gate * combined

        output = self.fc(fused_features)

        return output

In [37]:
fusion = GatedFusionAdd2(512)

In [41]:
embedding_out_imu = embedding_out_imu.permute(0, 2, 1)
embedding_out_wifi = embedding_out_wifi.permute(0, 2, 1)

In [42]:
embedding_out_imu.shape

torch.Size([4, 256, 512])

In [43]:
fusion_out = fusion(embedding_out_imu, embedding_out_wifi)

hihihi torch.Size([4, 256, 1024])


In [44]:
fusion_out.shape

torch.Size([4, 256, 512])

## 测试文本编码模块

### 一些小尝试

In [11]:
import torch
from transformers import AutoTokenizer, AutoModel


model_path = "D:\MyCodes\CLIPBased_TAD\model\openai\clip-vit-base-patch32"

In [12]:
from transformers import CLIPProcessor, CLIPModel, AutoTokenizer, AutoProcessor

clip_processor = CLIPProcessor.from_pretrained(model_path)
clip_encoder = CLIPModel.from_pretrained(model_path).requires_grad_(False)

In [17]:
label[0]

tensor([[0.0000, 0.2333, 7.0000],
        [0.2333, 0.6000, 1.0000],
        [0.6000, 1.0000, 5.0000]])

In [13]:
from dataset.action import id_to_attribute


In [14]:
action_label_idx = label[0][0][2].item()

In [15]:
id_to_attribute[action_label_idx]

{'hand_action': 'Lifting cup to lips, swallowing',
 'torso_action': 'Head tilted back',
 'leg_action': 'Stationary standing/sitting'}

In [16]:
text_list = []

for k in id_to_attribute[action_label_idx]:
    text_list.append(id_to_attribute[action_label_idx][k])

text_list


['Lifting cup to lips, swallowing',
 'Head tilted back',
 'Stationary standing/sitting']

In [17]:
text_input =  clip_processor(text=text_list, return_tensors="pt", padding=True)

In [18]:
text_input['input_ids']


tensor([[49406, 10857,  1937,   531,  8847,   267, 13433,  8283, 49407],
        [49406,  1375,  6124,   775,   893, 49407, 49407, 49407, 49407],
        [49406, 29493,  2862,   270,  4919, 49407, 49407, 49407, 49407]])

In [19]:
text_embeds = clip_encoder.get_text_features(**text_input)
text_embeds.shape

torch.Size([3, 512])

In [20]:
text_embeds

tensor([[ 0.1980,  0.1285, -0.2894,  ...,  0.2810,  0.0626, -0.5619],
        [-0.0973, -0.3018, -0.3150,  ..., -0.4032, -0.0276, -0.3218],
        [ 0.0234, -0.2164,  0.2205,  ..., -0.1949, -0.0291, -0.4148]])

In [21]:
from model.VisionTransformer import VisionTransformer, ViT_wo_patch_embed, MB_ViT_v3, MB_ViT_v3_shareweight
from functools import partial
import torch.nn as nn

text_self_attention = ViT_wo_patch_embed(global_pool=False, embed_dim=512, depth=1,
                                                  num_heads=4, mlp_ratio=4, qkv_bias=True,
                                                  norm_layer=partial(nn.LayerNorm, eps=1e-6))  # in: B*L*C

In [22]:
text_embeds = text_embeds.unsqueeze(0) 
text_embeds.shape

torch.Size([1, 3, 512])

In [25]:
text_embeds_att, _ = text_self_attention(text_embeds)
text_embeds_att.shape

torch.Size([1, 512])

### 将上述文本编码综合

In [11]:
from model.embeddings import TextEmbedding

In [12]:
emb = TextEmbedding()

In [13]:
label

[tensor([[ 0.0000,  0.2660, 23.0000],
         [ 0.2660,  0.4993, 24.0000],
         [ 0.4993,  0.7327,  7.0000],
         [ 0.7327,  1.0000, 22.0000]]),
 tensor([[ 0.1000,  0.4333, 13.0000],
         [ 0.4333,  0.8000, 20.0000],
         [ 0.8000,  1.0000, 24.0000]]),
 tensor([[0.2667, 0.6000, 1.0000],
         [0.6000, 1.0000, 6.0000]]),
 tensor([[ 0.0333,  0.3667, 20.0000],
         [ 0.3667,  0.5667, 14.0000],
         [ 0.5667,  0.8000,  3.0000],
         [ 0.8000,  1.0000,  4.0000]])]

In [14]:
out = emb(label)
out[0].shape

torch.Size([4, 512])

### 测试在anchor-free情况下的文本编码

In [5]:
from model.embeddings import TextEmbedding2

In [6]:
emb_free = TextEmbedding2()

In [7]:
out = emb_free(label)

In [8]:
out.shape

torch.Size([4, 2048, 512])

## CLIP LOSS

### 一些尝试

In [ ]:
import torch

B = 4
seq_len = 2048
d_model = 512

# for every batch 
text_emb = torch.rand(seq_len, d_model)
sig_emb = torch.rand(seq_len, d_model)

text_emb.shape

torch.Size([2048, 512])

In [ ]:
import torch
import torch.nn.functional as F
from torch import nn
import numpy as np
def get_logits( embeds1, embeds2, logit_scale):
    # 计算image_features @ text_features.T相似度矩阵
    logits_per_embeds1 = logit_scale * embeds1 @ embeds2.T
    logits_per_embeds2 = logit_scale * embeds2 @ embeds1.T
    return logits_per_embeds1, logits_per_embeds2

logit_scale = nn.Parameter(torch.ones([]) * np.log(1 / 0.07)).exp()

logits_per_embeds1, logits_per_embeds2 = get_logits(text_emb, sig_emb, logit_scale)
logits_per_embeds1.shape

def cal_clip_loss(image_features, text_features, logit_scale=logit_scale):
    device = image_features.device
    logits_per_image, logits_per_text = get_logits(image_features, text_features, logit_scale)
    labels = torch.arange(logits_per_image.shape[0], device=device, dtype=torch.long)
    total_loss = (
        F.cross_entropy(logits_per_image, labels) +
        F.cross_entropy(logits_per_text, labels)
    ) / 2

torch.Size([2048, 2048])

### 整合 anchor-free

In [5]:
import torch.nn as nn
import torch
import numpy as np


def clip_loss2(logits_per_image, logits_per_text):

    device = logits_per_image.device

    labels = torch.arange(logits_per_image.shape[0], device=device, dtype=torch.long)
    total_loss = (
        F.cross_entropy(logits_per_image, labels) +
        F.cross_entropy(logits_per_text, labels)
    ) / 2
    
    total_loss.to(device)

    # print("total_loss: ", total_loss.device.type)
    # print(device)
    # assert total_loss.device.type == device, "total_loss"
    

    return total_loss

In [25]:
from model.TAD.embedding import Embedding
from model.XRF.fusion import GatedFusionAdd2
from model.embeddings import TextEmbedding2

class CLIP(nn.Module):
    def __init__(self, device='cuda'):
        super(CLIP, self).__init__()
        self.device = device

        self.embedding_wifi = Embedding(in_channels=270, out_channels=512)
        self.embedding_imu = Embedding(in_channels=30, out_channels=512)
        self.fusion = GatedFusionAdd2(512)

        self.embedding_text = TextEmbedding2(device=self.device)
        # 冻结所有TextEmbedding2的参数
        for param in self.embedding_text.parameters():
            param.requires_grad = False

        self.logit_scale = nn.Parameter(torch.ones([]) * np.log(1 / 0.07))
        # self.logit_scale = torch.ones([])

        
        
        self.to(self.device)
        self.embedding_text.to(self.device)

    
    def forward(self, wifi, imu, text, status='train'):
        wifi_embeds = self.embedding_wifi(wifi)
        imu_embeds = self.embedding_imu(imu)

        # wifi_embeds, imu_embeds = wifi_embeds.to(self.device), imu_embeds.to(self.device) # to debug

        wifi_embeds = wifi_embeds.permute(0, 2, 1)
        imu_embeds = imu_embeds.permute(0, 2, 1)
        sig_embds = self.fusion(wifi_embeds, imu_embeds) # B * 2048 * 512

        if status == 'train':
            text_embeds = self.embedding_text(text) # B * 2048 * 512
        if status == 'test':
            text_embeds = text
        
        # text_embeds = text_embeds.to(self.device) # to debug
 
        sig_embds = sig_embds.reshape(-1, 512) # (B * 2048 )* 512
        text_embeds = text_embeds.reshape(-1, 512) # (B * 2048 )* 512

        # sig_embds.to(self.device)
        assert sig_embds.device.type == 'cuda', "sig输入数据未迁移"
        # text_embeds = text_embeds.to(self.device)
        assert text_embeds.device.type == self.device, "text输入数据未迁移"

        # 归一化 ？ to understand
        sig_embds = sig_embds / sig_embds.norm(dim=1, keepdim=True)  # [batch_img,512]
        text_embeds = text_embeds / text_embeds.norm(dim=1, keepdim=True)  # [batch_text,512]

        # 相似度计算
        logits_per_sig = self.logit_scale * sig_embds @ text_embeds.T
        logits_per_text = self.logit_scale * text_embeds @ sig_embds.T # (B * 2048) * (B * 2048)

        assert logits_per_sig.device.type == self.device, "logits_per_sig"
        assert logits_per_text.device.type == self.device, "logits_per_text"

        

        return logits_per_sig, logits_per_text


In [ ]:
import torch

from tqdm.auto import tqdm
from typing import Dict, List, Tuple

def train_step(model: torch.nn.Module, 
               dataloader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               optimizer: torch.optim.Optimizer,
               device: torch.device):
    
    model.train()
    train_loss = 0.0
    for batch, (X, y) in enumerate(dataloader):
        # 确保数据移至设备（若需要）
        wifi = X['wifi']
        imu = X['imu']
        wifi, imu = wifi.to(device), imu.to(device)
        text = [tensor.to(device, non_blocking=True) for tensor in y]
        # X, y = X.to(device), y.to(device)
        # for i, tensor in enumerate(text):
        #   print(f"Tensor {i} is on device: {tensor.device}")

        assert wifi.device.type == device, "wifi"
        assert imu.device.type == device, "imu"
        # assert text.device.type == 'cuda', "text"
        
        # 前向传播
        sig_emb, text_emb = model(wifi, imu, text)
        tmp_sig_emb, tmp_text_emb = sig_emb, text_emb

        # intermediate_outputs = y_pred.clone() 
        # 计算损失
        loss = loss_fn(tmp_sig_emb, tmp_text_emb)
        assert loss.device.type == device, "loss未迁移"
        
        train_loss += loss.item()  # 累加损失
        
        # 反向传播
        
        optimizer.zero_grad()
        
        # loss.backward(retain_graph=True)  # 移除retain_graph=True
        
        # for name, param in model.named_parameters():
        #     print(f"{name}梯度设备: {param.device.type}") 


        loss.backward(retain_graph=True)

        # for name, param in model.named_parameters():
        #     if param.grad is not None:
        #         print(f"{name}有梯度设备: {param.grad.device}")  # 应该与模型参数设备一致
        #     else:
        #         print(f"{name}无梯度")  # 可能未参与计算

        # 检查所有参数的梯度设备
        # for name, param in model.named_parameters():
        #     if param.grad is not None:
        #         print(f"{name}梯度设备: {param.grad.device}")  # 应该与模型参数设备一致
        #     else:
        #         print(f"{name}无梯度")  # 可能未参与计算

        optimizer.step()

        print("succeed!")

    # 计算整个epoch的平均损失
    train_loss /= len(dataloader)
    return train_loss

def test_step(model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
    
    pass

def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:

  # Create empty results dictionary
  results = {"train_loss": [],
  }

  torch.cuda.empty_cache()

  # Loop through training and testing steps for a number of epochs
  for epoch in tqdm(range(epochs)):
        train_loss = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)


        # 保存训练的模型
        model_json = model.to_json()
        with open('./save/model.json', 'w') as file:
            file.write(model_json)
        # 保存训练的权重
        model.save_weights('./save/model.h5')

      # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
      )

      # Update results dictionary
        results["train_loss"].append(train_loss)


  # Return the filled results at the end of the epochs
  return results

In [ ]:
import torch.optim as optim
import torch.nn.functional as F

device = 'cuda'  

net = CLIP(device)

optimizer = optim.AdamW(
    net.parameters(),
    lr=5e-4,          # 学习率
    weight_decay=0.1, # 权重衰减
    betas=(0.9, 0.999) # beta1 和 beta2
)

train(net , train_data_loader, optimizer, clip_loss2, 10, device)


  0%|          | 0/10 [00:00<?, ?it/s]

hihihi
hihihihi
succeed!
hihihi
hihihihi
succeed!
hihihi
hihihihi
succeed!
hihihi
hihihihi


In [12]:
from model.embeddings import getTextEmbeddings

In [13]:
text  = getTextEmbeddings()

In [14]:
text.shape

torch.Size([90, 512])

In [11]:
torch.save(net.state_dict(), r'D:/MyCodes/CLIPBased_TAD/save/model_state_dict.pth')

In [22]:
wifi = data['wifi'][0].to('cuda')  # 将 wifi 数据移动到 GPU
imu = data['imu'][0].to('cuda')    # 将 imu 数据移动到 GPU
text = text.to('cuda')          # 将 text 数据移动到 GPU

wifi = wifi.unsqueeze(0)
imu = imu.unsqueeze(0)

out1, out2 = net(wifi, imu, text, 'test')

logits_per_sig:  cuda
logits_per_text:  cuda


In [23]:
out1.shape

torch.Size([2048, 90])

In [24]:
out1

tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], device='cuda:0',
       grad_fn=<MmBackward0>)

### 基于 anchor-based 